# Built-in tools: RAG + Code Interpreter


1. **File Search (RAG)**: Search within an Apollo 14 document.
2. **Code Interpreter**: Calculate asteroid crater size.

> ⚠️ **Important**: File search requires a **local model deployment** in the AI Foundry project. 
> Unlike other labs that can use only the APIM gateway, File search needs direct model access 
> for its internal processing. This lab deploys a local `gpt-4.1` model alongside the APIM connection.

## Setup
Initialize the client and ensure resources exist.

In [12]:
!pip install azure-ai-projects==2.0.0b2 -q


[notice] A new release of pip is available: 25.2 -> 25.3
[notice] To update, run: pip install --upgrade pip


In [ ]:
import subprocess

# Configuration
RESOURCE_GROUP = "foundry-builtin-tools-rg"
LOCATION = "swedencentral"

# Check if resource group exists
result = subprocess.run(
    f'az group exists -n {RESOURCE_GROUP}',
    shell=True, capture_output=True, text=True
)

if result.stdout.strip() == 'false':
    print(f"📦 Creating resource group: {RESOURCE_GROUP}")
    subprocess.run(f'az group create -n {RESOURCE_GROUP} -l {LOCATION}', shell=True, check=True)
else:
    print(f"📦 Resource group exists: {RESOURCE_GROUP}")

# Get deployer principal ID for RBAC
deployer_id = subprocess.run(
    'az ad signed-in-user show --query id -o tsv',
    shell=True, capture_output=True, text=True
).stdout.strip()
print(f"👤 Deployer ID: {deployer_id}")

In [ ]:
%%bash -s "$RESOURCE_GROUP" "$deployer_id"
# Deploy Infrastructure
az deployment group create --debug \
    -g $1 \
    -n spoke \
    -f spoke.bicep \
    -p deployerPrincipalId=$2 \
    --query "properties.outputs" \
    -o json

In [ ]:
import subprocess, json, time, os
from azure.ai.projects import AIProjectClient
# Added CodeInterpreterToolAuto
from azure.ai.projects.models import PromptAgentDefinition, FileSearchTool, CodeInterpreterTool, CodeInterpreterToolAuto
from azure.identity import DefaultAzureCredential

# Deployment Config
RESOURCE_GROUP = "foundry-builtin-tools-rg"

# Get outputs
outputs = json.loads(subprocess.run(
    f'az deployment group show -g {RESOURCE_GROUP} -n spoke --query properties.outputs -o json',
    shell=True, capture_output=True, text=True
).stdout)

PROJECT_ENDPOINT = outputs['projectEndpoint']['value']
MODEL_DEPLOYMENT = outputs['modelDeploymentName']['value']

# Set environment variables for the SDK to use
os.environ["AZURE_AI_MODEL_DEPLOYMENT_NAME"] = MODEL_DEPLOYMENT
os.environ["FOUNDRY_PROJECT_ENDPOINT"] = PROJECT_ENDPOINT

project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT, 
    credential=DefaultAzureCredential()
)

# Get the OpenAI-compatible client for agent operations
openai_client = project_client.get_openai_client()

print(f"✅ Connected to Project: {PROJECT_ENDPOINT}")
print(f"✅ Model Deployment: {MODEL_DEPLOYMENT}")

## 1. File Search (RAG)
We'll download an Apollo 14 document and ask the agent about the "abort switch".

In [18]:
# Download Apollo 14 Flight Journal Index
!wget -q -O apollo14.html https://apollojournals.org/afj/ap14fj/19_day5_troubleshooting_lm_computer.html
print("✅ Downloaded apollo14.html")

# Create Vector Store
vector_store = openai_client.vector_stores.create(name="apollo-store")
print(f"✅ Vector Store: {vector_store.id}")

# Upload file to vector store (handles upload & poll)
with open("apollo14.html", "rb") as f:
    file = openai_client.vector_stores.files.upload_and_poll(
        vector_store_id=vector_store.id, 
        file=f
    )
print(f"✅ File Uploaded & Indexed: {file.id}")

# Create Agent with File Search Tool
with project_client:
    rag_agent = project_client.agents.create_version(
        agent_name="apollo-historian",
        definition=PromptAgentDefinition(
            model=MODEL_DEPLOYMENT,
            instructions="You are an Apollo expert. Use the file search tool to answer questions. Give exact citations.",
            tools=[FileSearchTool(vector_store_ids=[vector_store.id])],
        ),
        description="Apollo history expert"
    )

print(f"✅ Agent Created: {rag_agent.name} (v{rag_agent.version})")

✅ Downloaded apollo14.html
✅ Vector Store: vs_ecoYzpj2PfmOiKymh0pZg7V7
✅ File Uploaded & Indexed: assistant-CsS1UJ6e8tL1ksCuXUGCqj
✅ Agent Created: apollo-historian (v1)


In [19]:
# Create Conversation
conversation = openai_client.conversations.create()
print(f"🧵 Conversation Created: {conversation.id}")

# Send Query
# Asking a specific technical question found in the transcript
question = "What verb and noun command did Houston ask Antares to enter to check the status of the abort bit?"
print(f"❓ Asking: '{question}'")

response = openai_client.responses.create(
    conversation=conversation.id,
    input=question,
    extra_body={
        "agent": {
            "name": rag_agent.name, 
            "version": rag_agent.version,
            "type": "agent_reference"
        }
    },
)

print(f"\n🤖 Answer:\n{response.output_text}")

# Cleanup
# project_client.agents.delete_version(rag_agent.name, rag_agent.version)

🧵 Conversation Created: conv_e1b41a02e6c9680800NgeGI6ljfTBeYbI7F1DtIzRz98HAHaNP
❓ Asking: 'What verb and noun command did Houston ask Antares to enter to check the status of the abort bit?'

🤖 Answer:
Houston instructed Antares to enter the command "Verb 11 Noun 10 Enter; 30 Enter" to check the status of the abort bit on Channel 30.


## 2. Crater Calculator (Code Interpreter)

In [20]:
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT, 
    credential=DefaultAzureCredential()
)

# Get the OpenAI-compatible client for agent operations
openai_client = project_client.get_openai_client()

In [21]:
from azure.ai.projects.models import CodeInterpreterToolAuto

INSTRUCTIONS = """
Calculate crater diameter (km) using Python.
Formula: D = 1.161 * (2700/2500)^0.33 * d^0.78 * v^0.44 * 9.81^(-0.22) * sin(45deg)^0.33 / 1000
where: d=diameter(m), v=velocity(m/s)

After executing the code with python,
return ONLY with the final number in km.
"""

# Create Agent with Code Interpreter
with project_client:
    code_agent = project_client.agents.create_version(
        agent_name="crater-calculator",
        definition=PromptAgentDefinition(
            model=MODEL_DEPLOYMENT,
            instructions=INSTRUCTIONS,
            tools=[CodeInterpreterTool(container=CodeInterpreterToolAuto(file_ids=[]))],
        ),
        description="Calculates crater sizes"
    )

print(f"✅ Agent Created: {code_agent.name} (v{code_agent.version})")

✅ Agent Created: crater-calculator (v1)


In [22]:
import re
from IPython.display import display, HTML

query = "Asteroid size: 500m, Velocity: 20km/s"

# Create Conversation
conversation_calc = openai_client.conversations.create()
print(f"🧵 Conversation Created: {conversation_calc.id}")

print("❓ Sending query...")
response = openai_client.responses.create(
    conversation=conversation_calc.id,
    input=query,
    extra_body={
        "agent": {
            "name": code_agent.name, 
            "version": code_agent.version, 
            "type": "agent_reference"
        }
    },
)

response_text = response.output_text
print(f"\n🤖 Result: {response_text}")

# Display the generated Python code from Code Interpreter
print("\n" + "="*60)
print("📝 Generated Python Code:")
print("="*60)
for output_item in response.output:
    if output_item.type == "code_interpreter_call":
        # Display code with syntax highlighting
        code_html = f'''
<div style="background:#1e1e1e; padding:16px; border-radius:8px; margin:10px 0; border:1px solid #3c3c3c;">
<pre style="color:#d4d4d4; margin:0; white-space:pre-wrap; font-family:'Consolas','Monaco',monospace; font-size:13px;">{output_item.code}</pre>
</div>
'''
        display(HTML(code_html))
print("="*60)

# Visualize
try:
    if response_text:
        numbers = re.findall(r"\d+\.?\d*", response_text)
        crater_km = float(numbers[-1]) if numbers else None
        if crater_km:
            from crater_physics import validate_crater
            from crater_viz import visualize_crater
            
            val = validate_crater(crater_km, 500, 20)
            visualize_crater(crater_km, val)
        else:
            print("No number found in result.")
    else:
        print("No result text returned.")

except Exception as e:
    print(f"Could not visualize: {e}")

🧵 Conversation Created: conv_2c369d81fe76fb7600ge0tv4Bs9ttX6jH1cDSMN7fhRXQBdGMi
❓ Sending query...

🤖 Result: 6.39

📝 Generated Python Code:
